# 多输入多输出通道

实现一下多输入通道互相关运算

In [1]:
import sys
sys.path.append('..')

In [2]:
from d2l import mindspore as d2l

def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

[WARNING] CORE(346116,ffffbb048640,python):2025-12-14-23:31:22.937.692 [mindspore/core/utils/ms_context.cc:533] GetJitLevel] Set jit level to O2 for rank table startup method.
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'n

验证互相关运算的输出

In [3]:
X = d2l.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = d2l.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

[WARNING] DEVICE(346116,ffffbb048640,python):2025-12-14-23:31:29.110.347 [mindspore/ccsrc/plugin/ascend/res_manager/mem_manager/ascend_memory_adapter.cc:127] Initialize] Free memory size is less than half of total memory size.Device 0 Device MOC total size:31675383808 Device MOC free size:1905336320 may be other processes occupying this card, check as: ps -ef|grep python


Tensor(shape=[2, 2], dtype=Float32, value=
[[ 5.60000000e+01,  7.20000000e+01],
 [ 1.04000000e+02,  1.20000000e+02]])

计算多个通道的输出的互相关函数

In [4]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return d2l.stack([corr2d_multi_in(X, k) for k in K], 0)

K = d2l.stack((K, K + 1, K + 2), 0)
K.shape

(3, 2, 2, 2)

In [5]:
corr2d_multi_in_out(X, K)

Tensor(shape=[3, 2, 2], dtype=Float32, value=
[[[ 5.60000000e+01,  7.20000000e+01],
  [ 1.04000000e+02,  1.20000000e+02]],
 [[ 7.60000000e+01,  1.00000000e+02],
  [ 1.48000000e+02,  1.72000000e+02]],
 [[ 9.60000000e+01,  1.28000000e+02],
  [ 1.92000000e+02,  2.24000000e+02]]])

1x1卷积

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = d2l.reshape(X, (c_i, h * w))
    K = d2l.reshape(K, (c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = d2l.matmul(K, X)
    return d2l.reshape(Y, (c_o, h, w))

X = d2l.normal((3, 3, 3), 0, 1)
K = d2l.normal((2, 3, 1, 1), 0, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(d2l.reduce_sum(d2l.abs(Y1 - Y2))) < 1e-6